### key-value RDD transfrom & action

1. transform 

    groupbykey

    reducebykey

    mapvalues

    keys

    join(outer, inner)

2. action

    countbykey

In [28]:
from operator import add

In [1]:
from pyspark import SparkConf, SparkContext

conf = SparkConf().setMaster('local').setAppName('test2')
sc = SparkContext(conf = conf)

### ```groupByKey() large, small 구별조심```

In [17]:
x = sc.parallelize([('a', 1), ('b', 1), ('a', 1), ('b', 1), ('b', 2), ('a', 2), ('a', 2), ('b', 2)], 2)

y = x.groupByKey().collect()

y

[('b', <pyspark.resultiterable.ResultIterable at 0x21a1b7f25e0>),
 ('a', <pyspark.resultiterable.ResultIterable at 0x21a1b7ffd90>)]

전에 했던 groupby 와 마찬가지로 key 에 해당하는 0 번째 index (a, b)와 iterable 한 pyspark 의 내부 클래스가 나오게 된다.

In [23]:
list1 = [(i, list(j)) for (i,j) in y]

[('b', [1, 1, 2, 2]), ('a', [1, 1, 2, 2])]

위와 같이 key 에 해당하는 value 값들을 확인할 수 있다.

추가로 기본 파티셔닝을 i 개로 지정했어도 groupbykey(j) 로하면 j 로 지정된다.

결과는 아래와 같다.

In [27]:
x.getNumPartitions(), x.groupByKey(3).getNumPartitions()

(2, 3)

### ```reduceByKey()``` 

In [29]:
x.reduceByKey(add).collect()

[('b', 6), ('a', 6)]

전에 [1,2,3,4,5] 라는 RDD 가 있을 때 RDD.reduce(add) 을 하면 요소들이 더하기로 합쳐져 하나의 결과값으로 출력되는 것을 배웠는데,

reducebykey 역시도 그렇다. 차이점은 key 를 기준값으로 하고 그 key 에 대한 value 들이 더해진다.

In [30]:
x.reduceByKey(lambda x: x%2).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 11.0 failed 1 times, most recent failure: Lost task 0.0 in stage 11.0 (TID 21) (192.168.124.100 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\Spark\spark-3.2.4-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 619, in main
  File "C:\Spark\spark-3.2.4-bin-hadoop2.7\python\lib\pyspark.zip\pyspark\worker.py", line 609, in process
  File "c:\Users\admin\anaconda3\envs\newkaf\lib\site-packages\pyspark\rdd.py", line 2918, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "c:\Users\admin\anaconda3\envs\newkaf\lib\site-packages\pyspark\rdd.py", line 417, in func
    return f(iterator)
  File "c:\Users\admin\anaconda3\envs\newkaf\lib\site-packages\pyspark\rdd.py", line 2146, in combineLocally
    merger.mergeV

위의 코드는 키별로 여러 값을 가지는 RDD를 단일 값으로 줄일 수 없기 때문에 오류가 발생한다.

태생자체가 reduce 이고 reduce 의 역할은 단일값으로 만드는 역할임을 기억하자.

그리고 reducebykey 의 개념자체는 사실 groupByKey + reduction(reduce) 랑 차이가 없다.

In [62]:
[(i, sum(list(j))) for (i, j) in y]

[('b', 6), ('a', 6)]

In [63]:
x.reduceByKey(lambda x, y: x + y).collect()

[('b', 6), ('a', 6)]

위와 같이 groupbykey + reduction(add) 를 하나 reducebykey 를 하나 결과값은 차이가 없다.

같은 개념이기 때문에 당연한 결과지만 속도는 reducebykey 가 더 빠르다고 한다.

### ```mapValues()```

In [68]:
rdd2 = sc.parallelize([('a', ['A', 'B', 'C']), ('b', ['E', 'F', 'G'])])

rdd2.mapValues(lambda x:len(x)).collect()

[('a', 3), ('b', 3)]

위의 코드와 같이 RDD 의 파티션과 key 를 제외한 value 값에만 함수를 적용시킨다. 

value 값에 길이를 반환하는 len 함수를 적용시키니 파티션 key 값은 그대로고 value 에만 적용시킨 결과값을 반환한다.

value 값에만 연산을 하다보니 불필요한 resource 를 줄일 수 있겠다.

### ```countbykey()```

In [71]:
x.collect()

[('a', 1),
 ('b', 1),
 ('a', 1),
 ('b', 1),
 ('b', 2),
 ('a', 2),
 ('a', 2),
 ('b', 2)]

In [70]:
x.countByKey()

defaultdict(int, {'a': 4, 'b': 4})

countByKey() 메소드는 위의 결과와 같이 각 key 들이 가진 value 의 개수를 count 한다.

'a', 'b' 라는 key 값에 해당하는 value 가 4 개가 있으니 4, 4 를 딕셔너리로 리턴한다.

In [73]:
x.countByKey().items()

dict_items([('a', 4), ('b', 4)])

In [74]:
type(x.countByKey()), type(x.countByKey().items())

(collections.defaultdict, dict_items)

위의 두 개의 결과 모두가 python object 임을 확인할 수 있다.

### ```keys()```

In [75]:
x.keys()

PythonRDD[101] at RDD at PythonRDD.scala:53

countbykey 는 action 이였지만 그냥 keys 는 transformation 에 해당한다는 사실을 알 수 있다.

파티션을 유지하거나, key 가 굉장히 많을 때 새로운 RDD 로 transform 하기위해 사용하면 되겠다.

In [78]:
x.keys().collect()

['a', 'b', 'a', 'b', 'b', 'a', 'a', 'b']

In [79]:
x.keys().distinct().count()

2

keys 는 별로 어려울 것이 없다.

### ```joins()```

joins 에는 inner, outer 가 있다.

inner 는 교집합을 생각하면 된다.

outer 는 left, right 가 있다.

outer 는 한 쪽에는 데이터가 있고 다른 쪽은 없을 때 있는 쪽의 데이터를 출력해준다.

left outer 는 왼쪽 데이터를 모두 출력하고 오른쪽에 데이터가 없는 경우 None

right outer 는 오른쪽 데이터를 모두 출력하고 왼쪽에 데이터가 없는 경우 None

이다. 

full outer 는 union 그냥 전부 보여준다.

코드를 보며 알아보자.


In [82]:
tes1 = sc.parallelize([('a', 20), ('b', 30), ('c', 40)])
tes2 = sc.parallelize([('a', 30), ('b', 40), ('b', 50), ('c', 60), ('d', 20)])


tes1.join(tes2).collect()

[('b', (30, 40)), ('b', (30, 50)), ('c', (40, 60)), ('a', (20, 30))]

위의 결과와 같이 key 끼리 묶고, 그 key 값에 해당하는 value 들을 튜플로 묶는다.

'b' 의 경우 tes2 에 'b' key 를 가진 튜플이 2 개 있어서 연산이 두 번 진행된다. (30, 40) (30, 50)

'd' 의 경우 key 값은 tes2 에만 존재하기 때문에 join 하지 않는다.

In [83]:
tes1.leftOuterJoin(tes2).collect()

[('b', (30, 40)), ('b', (30, 50)), ('c', (40, 60)), ('a', (20, 30))]

RDD1.leftOuterJoin(RDD2).collect() 라고 했을 때 

항상 기준은 변수를 선언한 쪽이 왼쪽이라고 보면 되겠다.

그럼 왼쪽(RDD1)을 기준으로 a, b, c 값을 합치면 되니까 

('a', (20, 30)), 'b' 는 2 번 ('b', (30, 40)), ('b', (30, 50)), ('c', (40, 60)) 이렇게 되겠다.

In [84]:
tes1.rightOuterJoin(tes2).collect()

[('b', (30, 40)),
 ('b', (30, 50)),
 ('c', (40, 60)),
 ('d', (None, 20)),
 ('a', (20, 30))]

RDD1.rightOuterJoin(RDD2).collect() 라고 했을 때 

tes1 은 왼쪽 tes2 는 오른쪽인데 오른쪽  tes2 에 있는 값을 기준으로 합치게 되니까 

a  1 번 (20, 30)

b  2 번 ('b', (30, 40)), ('b', (30, 50))

c  1 번 ('c', (40, 60))

d  1 번 (None, 20) 

위와 같이 tes1 에는 d 라는 키가 없어서 value 가 없으므로 None 값이 끼워진다.

In [85]:
tes1.fullOuterJoin(tes2).collect()

[('b', (30, 40)),
 ('b', (30, 50)),
 ('c', (40, 60)),
 ('d', (None, 20)),
 ('a', (20, 30))]

fullouterjoin은 그냥 합집합이다. 

결과는 rightOuterJoin 과 같았지만 항상 그런 것은 아니다.

마지막 반례코드로 마무리 하겠습니다.

In [86]:
rdd_ = sc.parallelize([(1, "Alice"), (2, "Bob")])
rdd__ = sc.parallelize([(1, 25), (3, 30)])

rdd_.fullOuterJoin(rdd__).collect()

[(2, ('Bob', None)), (1, ('Alice', 25)), (3, (None, 30))]

In [87]:
rdd_.rightOuterJoin(rdd__).collect()

[(1, ('Alice', 25)), (3, (None, 30))]